<a href="https://colab.research.google.com/github/MANUPRIYASINGH/MSWC_FSCIL_Reproduction/blob/main/TCN_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neurobench

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 745.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  U

In [1]:
import copy
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, ConcatDataset

from neurobench.benchmarks import Benchmark
from neurobench.datasets import MSWC
from neurobench.datasets.MSWC_IncrementalLoader import IncrementalFewShot
from tqdm import tqdm

In [2]:
!git clone https://github.com/MANUPRIYASINGH/MSWC_FSCIL_Reproduction.git

Cloning into 'MSWC_FSCIL_Reproduction'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 32 (delta 8), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (32/32), 17.86 MiB | 13.67 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [4]:
cd MSWC_FSCIL_Reproduction/

/content/MSWC_FSCIL_Reproduction


In [5]:
!pip install git+https://github.com/V0XNIHILI/TCN-library.git

  Cloning https://github.com/V0XNIHILI/TCN-library.git to /tmp/pip-req-build-32fuoqpx
  Running command git clone --filter=blob:none --quiet https://github.com/V0XNIHILI/TCN-library.git /tmp/pip-req-build-32fuoqpx
  Resolved https://github.com/V0XNIHILI/TCN-library.git to commit a32c26e70cff873fb397871f7697ec56cc8d5116
  Preparing metadata (setup.py) ... done
  Created wheel for tcn-library: filename=tcn_library-0.0.1-py3-none-any.whl size=12922 sha256=273cd95894c974a40a98a63a2d7b9065592c5c91108a68180cd64a98b0ba9e54
  Stored in directory: /tmp/pip-ephem-wheel-cache-7_53ltlo/wheels/0c/56/1c/31842b9b5052d6df41c630521df68b0b12ddf10a0214d6766e
Successfully built tcn-library


In [6]:
import torch.optim as optim

In [7]:
# data in repo root dir
ROOT = "./data/"

directory = "./model_data"

if not os.path.exists(directory):
        try:
            os.makedirs(directory)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise

NUM_WORKERS = 4
BATCH_SIZE = 256

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == torch.device("cuda"):
    PIN_MEMORY = True
else:
    PIN_MEMORY = False
device

device(type='cpu')

In [9]:
SPIKING = False

In [10]:
from neurobench.preprocessing import MFCCPreProcessor, S2SPreProcessor

n_fft = 512
win_length = None
hop_length = 240
n_mels = 20
n_mfcc = 20

if SPIKING:
    encode = S2SPreProcessor(device, transpose=True)
    config_change = {"sample_rate": 48000,
                     "hop_length": 240}
    encode.configure(threshold=1.0, **config_change)
else:
    encode = MFCCPreProcessor(
        sample_rate=48000,
        n_mfcc=n_mfcc,
        melkwargs={
            "n_fft": n_fft,
            "n_mels": n_mels,
            "hop_length": hop_length,
            "mel_scale": "htk",
            "f_min": 20,
            "f_max": 4000,
        },
        device = device
    )

In [11]:
base_train_set = MSWC(root=ROOT, subset="base", procedure="training")

train_loader = DataLoader(base_train_set, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, shuffle=True)

base_validation_set = MSWC(root=ROOT, subset="base", procedure="validation")

validation_loader = DataLoader(base_validation_set, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, shuffle=True)

downloading ....


100%|██████████| 651349274/651349274 [00:03<00:00, 204118146.03it/s]


Unzipping file...


Extracting files: 100%|██████████| 90244/90244 [00:27<00:00, 3329.06file/s]


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [15]:
from tcn_lib import TCN
from torchsummary import summary

feature_count = 128
model = TCN(20, 200, [64] * 2 + [128] * 2, [9] * 4, batch_norm=True, weight_norm=True,
            residual=True, bottleneck=True, groups=32, dropout = 0.2).to(device)


summary(model, (20, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       _WeightNorm-1                [-1, 20, 1]               0
ParametrizedConv1d-2              [-1, 64, 200]           1,280
       _WeightNorm-3                [-1, 20, 1]               0
       _WeightNorm-4                [-1, 20, 1]               0
       _WeightNorm-5                [-1, 20, 1]               0
       _WeightNorm-6                [-1, 20, 1]               0
       BatchNorm1d-7              [-1, 64, 200]             128
              ReLU-8              [-1, 64, 200]               0
           Dropout-9              [-1, 64, 200]               0
      _WeightNorm-10                 [-1, 2, 9]               0
ParametrizedConv1d-11              [-1, 64, 208]           1,152
      _WeightNorm-12                 [-1, 2, 9]               0
      _WeightNorm-13                 [-1, 2, 9]               0
      _WeightNorm-14                 [

In [16]:
class SaveBestModel:
    """
    Class to save the best model while training. If the current epoch's
    validation loss is less than the previous least less, then save the
    model state.
    """
    def __init__(
        self, best_valid_loss= 100
    ):
        self.best_valid_loss = best_valid_loss

    def __call__(
        self, current_valid_loss,
        epoch, model
    ):
        if current_valid_loss < self.best_valid_loss:
            self.best_valid_loss = current_valid_loss
            print(f"\nLeast validation error: {self.best_valid_loss}")
            print(f"\nSaving best model for epoch: {epoch+1}\n")
            torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                }, './outputs/best_model.pth')

In [18]:
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm

# Define your hyperparameters
epochs = 15
lr_max = 5e-3  # Initial maximum learning rate
lr_min = 1e-5  # Minimum learning rate
T_max = epochs  # Number of epochs for one cycle

# Create loss function, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr_max)
lr_scheduler = CosineAnnealingLR(optimizer, T_max=T_max, eta_min=lr_min)
save_best_model = SaveBestModel()

for epoch in tqdm(range(epochs)):
    train_avg_loss = 0
    train_correct = 0
    train_total = 0

    validation_avg_loss = 0
    validation_correct = 0
    validation_total = 0

    model.train()

    for data, target in train_loader:
        data, target = encode((data.to(device), target.to(device)))
        data = data.squeeze()

        optimizer.zero_grad()

        output = model(data)  # Model returns only the output

        # Calculate Loss
        loss = criterion(output, target)

        loss.backward()
        optimizer.step()

        train_avg_loss += loss.item()
        _, predicted = output.max(1)
        train_total += target.size(0)
        train_correct += predicted.eq(target).sum().item()


    model.eval()

    with torch.no_grad():
        for data, target in validation_loader:
            data, target = encode((data.to(device), target.to(device)))
            data = data.squeeze()

            output = model(data)  # Model returns only the output
            loss = criterion(output, target)

            validation_avg_loss += loss.item()
            _, predicted = output.max(1)
            validation_total += target.size(0)
            validation_correct += predicted.eq(target).sum().item()

    train_loss, train_acc = train_avg_loss / len(train_loader), 100 * train_correct / train_total
    validation_loss, validation_acc = validation_avg_loss / len(validation_loader), 100 * validation_correct / validation_total

    print(f"Epoch {epoch} - Train Loss: {train_loss:.4f} - Train Acc: {train_acc:.2f}")
    print(f"Epoch {epoch} - Validation Loss: {validation_loss:.4f} - Validation Acc: {validation_acc:.2f}")

    save_best_model(
        validation_loss, epoch, model
    )

    # Step the scheduler
    lr_scheduler.step()

print('Finished Training')


  7%|▋         | 1/15 [17:17<4:02:06, 1037.63s/it]

Epoch 0 - Train Loss: 3.3356 - Train Acc: 17.97
Epoch 0 - Validation Loss: 2.0011 - Validation Acc: 45.04

Least validation error: 2.0011341363191604

Saving best model for epoch: 1



 13%|█▎        | 2/15 [34:45<3:46:04, 1043.40s/it]

Epoch 1 - Train Loss: 2.0108 - Train Acc: 44.42
Epoch 1 - Validation Loss: 1.2892 - Validation Acc: 64.26

Least validation error: 1.2891912907361984

Saving best model for epoch: 2



 20%|██        | 3/15 [52:07<3:28:35, 1042.93s/it]

...

Epoch 12 - Train Loss: 0.8685 - Train Acc: 74.97
Epoch 12 - Validation Loss: 0.5814 - Validation Acc: 84.00


 93%|█████████▎| 14/15 [4:03:25<17:23, 1043.24s/it]

Epoch 13 - Train Loss: 0.8554 - Train Acc: 75.34
Epoch 13 - Validation Loss: 0.5638 - Validation Acc: 84.35


100%|██████████| 15/15 [4:20:50<00:00, 1043.35s/it]

Epoch 14 - Train Loss: 0.8455 - Train Acc: 75.76
Epoch 14 - Validation Loss: 0.5509 - Validation Acc: 84.28

Least validation error: 0.5508516222238541

Saving best model for epoch: 15

Finished Training
